In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import string

import six # needed for Google Cloud client
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [2]:
# !aws s3 ls 

In [51]:
#!pip install --upgrade google-cloud-language

In [53]:
!pip install unidecode
import unidecode

In [3]:
!ls -lr recipeVectors

total 220788
-rw-rw-r-- 1 apanza apanza  8842166 Jul 27 02:17 us-recipes.json
-rw-rw-r-- 1 apanza apanza  5233713 Jul 27 02:17 trusted_brands.json
-rw-rw-r-- 1 apanza apanza     1920 Aug 11 00:33 test.py
-rw-rw-r-- 1 apanza apanza      270 Jul 23 20:20 scrapy.cfg
drwxrwxr-x 3 apanza apanza     4096 Aug 11 00:40 recipeVectors
drwxrwxr-x 2 apanza apanza     4096 Jul 23 20:20 recipes
-rw-rw-r-- 1 apanza apanza 14580388 Jul 24 13:46 allrecipes_world-cuisine.json
-rw-rw-r-- 1 apanza apanza 38022625 Aug 12 02:55 allRecipes_tony_categories.json
-rw-rw-r-- 1 apanza apanza  9471300 Jul 25 20:49 allRecipes_recipes_seafood_and_sidedish.json
-rw-rw-r-- 1 apanza apanza 21539225 Jul 26 05:06 allRecipes_recipes_meatpoultry_and_pastanoodles.json
-rw-rw-r-- 1 apanza apanza 19410068 Jul 25 14:05 allRecipes_recipes_lunch_and_maindish.json
-rw-rw-r-- 1 apanza apanza 13800293 Aug 12 19:58 allRecipes_recipes.json
-rw-rw-r-- 1 apanza apanza 18675330 Jul 24 18:27 allRecipes_recipes_healthy_and_holidays.json
-

In [4]:
df = pd.read_json('./recipeVectors/allRecipes_recipes.json', dtype={})
# df2 = pd.read_json('./data/allRecipes_recipes_bbq_bread.json', dtype={})

# df['category'] = 'salads'
# df2['category'] = 'bbqbread'
# df_merged = pd.concat([df,df2])
print len(df)
# print len(df2)
# print len(df_merged)
df.sample(5)

10095


,categories,cookingTime,description,ingredients,instructionSteps,name,rating,ratingCount,url
5216,"[Fish Recipes, Everyday Cooking, Main Dishes, ...",PT43M,NaN,"[4 pounds bluefish fillets, 1 cup orange juice...",[Place the fish fillets into a large bowl. Pou...,Super Summer Grilled Bluefish,4.67,23,http://allrecipes.com/recipe/87237/super-summe...
8352,"[Shrimp Recipes, Shrimp Scampi, Everyday Cooki...",PT40M,NaN,"[1/3 cup butter, 1/3 cup olive oil, 2 tablespo...",[Combine the butter and the olive oil in a lar...,My Special Shrimp Scampi Florentine,4.64,84,http://allrecipes.com/recipe/156232/my-special...
773,"[Vegetable Salads, Salad Recipes, Vegan Recipe...",PT55M,NaN,"[1 eggplant - quartered lengthwise, and sliced...",[Preheat the oven to 450 degrees F (230 degree...,Emily's Famous Roasted Vegetable Salad,3.29,14,http://allrecipes.com/recipe/61680/emilys-famo...
4389,"[Onion Soup, Vegetable Soup, Soup, Soups, Stew...",PT1H20M,NaN,"[2 tablespoons olive oil, 8 large red onions, ...",[Heat olive oil in a Dutch oven or soup pot ov...,Real French Onion Soup,3.45,10,http://allrecipes.com/recipe/72134/real-french...
9124,"[Bean and Pea Soups, Soup, Soups, Stews and Ch...",NaN,NaN,"[1 pound dry navy beans, soaked overnight, 4 q...","[In a large stock pot, add water, ham bone and...",Ham and Bean Soup II,4.44,125,http://allrecipes.com/recipe/13380/ham-and-bea...


In [63]:
df2 = pd.read_json('./recipeVectors/allRecipes_tony_categories.json', dtype={})
print len(df2)

28877


In [7]:
from collections import defaultdict
import operator

categories = defaultdict(int)
for idx, row in df.iterrows():
    try:
        for cat in row['categories']:
            categories[cat] += 1 
    except TypeError:
        # for cases where there is no category found, row['categories'] is of type float and value nan
        pass
print 'Top 30 Categories:'
for item in sorted(categories.items(), reverse=True, key=operator.itemgetter(1))[:30]:
    print item

Top 30 Categories:
(u'Recipes', 9893)
(u'Everyday Cooking', 7463)
(u'Soups, Stews and Chili', 4014)
(u'Salad Recipes', 3380)
(u'Soup', 2730)
(u'Main Dishes', 1307)
(u'Vegetable Soup', 1189)
(u'Vegetable Salads', 911)
(u'Canned Beans and Peas', 894)
(u'Pasta by Shape', 862)
(u'Asian Recipes', 732)
(u'Stew', 696)
(u'Side Dishes', 650)
(u'Fish Recipes', 635)
(u'Canned Vegetables', 576)
(u'Chicken Breasts', 546)
(u'Vegetable Recipes', 543)
(u'Summer Salads', 516)
(u'Vegan Recipes', 502)
(u'Pasta Salad', 479)
(u'Chili Powder', 474)
(u'Chili', 472)
(u'Cream Soups', 463)
(u'Green Salads', 450)
(u'Mexican Recipes', 421)
(u'Winter Soups and Stews', 410)
(u'Shrimp Recipes', 386)
(u'Italian Recipes', 380)
(u'Lettuce Recipes', 370)
(u'Greens Recipes', 358)


In [8]:
import re

def convertCookingTimeToMinutes(text):
    text = text.replace('PT','')
    arr = re.findall(r'[0-9]+', text)
    minute = 0
    if len(arr) == 2:
        minute = int(arr[0])* 60 + int(arr[1])
    if len(arr) == 1:
        if 'M' in text:
            minute = int(arr[0])
        if 'H' in text:
            minute = int(arr[0]) * 60
    return minute
df_merged['cookingTimeMinutes'] = df_merged['cookingTime'].apply(convertCookingTimeToMinutes)
df_merged[['cookingTime','cookingTimeMinutes']].sample(5)

NameError: name 'df_merged' is not defined

In [9]:
pd.set_option('display.max_colwidth', -1)
df.iloc[[10]]['ingredients']

10    [2 skinless, boneless chicken breast halves, 1 (1.27 ounce) packet dry fajita seasoning, divided, 1 tablespoon vegetable oil, 1 (15 ounce) can black beans, rinsed and drained, 1 (11 ounce) can Mexican-style corn, 1/2 cup salsa, 1 (10 ounce) package mixed salad greens, 1 onion, chopped, 1 tomato, cut into wedges, Add all ingredients to list, Add all ingredients to list]
Name: ingredients, dtype: object

In [10]:
df[['ingredients']].values[10][0]

[u'2 skinless, boneless chicken breast halves',
 u'1 (1.27 ounce) packet dry fajita seasoning, divided',
 u'1 tablespoon vegetable oil',
 u'1 (15 ounce) can black beans, rinsed and drained',
 u'1 (11 ounce) can Mexican-style corn',
 u'1/2 cup salsa',
 u'1 (10 ounce) package mixed salad greens',
 u'1 onion, chopped',
 u'1 tomato, cut into wedges',
 u'Add all ingredients to list',
 u'Add all ingredients to list']

In [99]:
def translate_non_alphanumerics(to_translate, translate_to=u''):
    # https://stackoverflow.com/a/1324274/2491761
    not_letters = u'!"#%\'()&*+,-./:;<=>?@[\]^_`{|}~0123456789'
    if isinstance(to_translate, unicode):
        translate_table = dict((ord(char), unicode(translate_to))
                               for char in not_letters)
    else:
        assert isinstance(to_translate, str)
        translate_table = string.maketrans(not_letters,
                                           translate_to
                                              *len(not_letters))
    return to_translate.translate(translate_table)

def extract_ingredients(full_ingred_list):
    # From: https://en.wikibooks.org/wiki/Cookbook:Units_of_measurement
    meas_units = [
        'teaspoon', 'teaspoons',
        't',
        'tsp', 'tsps',
        'tablespoon','tablespoons',
        'tbl',
        'tbs',
        'tbsp', 'tbsps',
        'fl', 'fluid',
        'oz', 'ozs',
        'ounce', 'ounces',
        'cup', 'cups',
        'c',
        'pint', 'pints',
        'p',
        'pt',
        'quart', 'quarts',
        'qt', 'qts',
        'q', 'qs',
        'gallon', 'gallons',
        'gal', 'gals',
        'ml',
        'milliliter', 'milliliters',
        'millilitre', 'millilitres',
        'cc',
        'l',
        'liter', 'liters',
        'litre', 'litres',
        'pinch', 'pinches',        
        'pound', 'pounds',
        'lb', 'lbs',
        'mg', 'mgs',
        'milligram', 'milligrams',
        'milligramme', 'milligrammes',
        'g', 'gs',
        'gram', 'grams',
        'gramme', 'grammes',
        'kg', 'kgs',
        'kilogram', 'kilograms',
        'kilogramme', 'kilogrammes',
        'mm', 'mms',
        'millimeter', 'millimeters',
        'millimetre', 'millimetres',
        'cm', 'cms',
        'centimeter', 'centimeters',
        'centimetre', 'centimetres',
        'm', 'ms',
        'meter', 'meters',
        'metre', 'metres',
        'inch', 'inches',
        'in', 'ins',
        ###
        'loaf', 'loaves',
        'pouch', 'pouches',
        'wedge', 'wedges',
        'drop', 'drops',
        'amount', 'amounts',
        'bulk', 'bulks',
        'coating', 'coatings',
        'carton', 'cartons',
        'count',
        'dusting', 'dustings',
        'roll', 'rolls',
        'hint', 'hints',
        'round', 'rounds',
        'cube', 'cubes',
        'husk', 'husks',
        'envelope', 'envelopes',
        'container', 'containers',
        'dash', 'dashes',
        'bitesize', 'bitesized',
        'bite', 'sized',
        'size',
        'each',
        'taste', 'desired',
        'can', 'cans',
        'unit', 'units',
        'box', 'boxes',
        'tub', 'tubs',
        'slab', 'slabs',
        'sprig', 'sprigs',
        'stalk', 'stalks',
        'matchstick', 'matchsticks',
        'balls',
        'clove', 'cloves',
        'slice', 'slices',
        'head', 'heads',
        'spear', 'spears',
        'chunk', 'chunks',
        'piece', 'pieces',
        'jar', 'jars',
        'package', 'packages',
        'pack', 'packs',
        'packet', 'packets',
        'bunch', 'bunches',
        'tube', 'tubes',
        'jug', 'jugs',
        'bottle', 'bottles',
        'stick', 'sticks',
        'strip', 'strips',
        'bag', 'bags',
        'dash', 'dashes',
        'container', 'containers',
        'envelope', 'envelopes',
        'rounds',
        'sheet', 'sheets',
        'squares',
        'whole',
        'extra', 'extras',
        'dozen', 'dozens',
        'half', 'halves',
        'third', 'thirds',
        'quarter', 'quarters',
        'fifth', 'fifths',
        'eighth', 'eighths',
    ]
    
    preparatory_descriptions = [
        'baked',
        'beat', 'beaten', 'wellbeaten',
        'blanched',
        'blended',
        'boiled',
        'bottled',
        'broiled',
        'broken',
        'browned',
        'canned',
        'chilled',
        'chipped',
        'chopped',
        'cleaned',
        'converted',
        'cooked',
        'cooled',
        'cored',
        'crumbled',
        'crushed',
        'cubed',
        'cut',
        'defrosted',
        'deshelled',
        'desilked',
        'deveined',
        'diced', 'dice',
        'dissolved',
        'divided',
        'drained',
        'dried',
        'filleted',
        'filtered',
        'flaked',
        'fresh',
        'fried',
        'frozen',
        'grated',
        'grilled',
        'ground',
        'halved',
        'hardened',
        'heated',
        'hulled',
        'husked',
        'jarred',
        'juiced',
        'julienned', 'julienne', 'juliennecut', 'juliennesliced',
        'kneaded', 'kneading',
        'marinated',
        'mashed',
        'matchstickcut',
        'melted',
        'microwaved',
        'minced',
        'mixed',
        'opened',
        'packed',
        'peeled',
        'pitted',
        'prepared',
        'pressed',
        'prewashed',
        'processed',
        'quartered',
        'raw',
        'refrigerated',
        'removed',
        'rinsed',
        'ripe',
        'ripened',
        'ripped',
        'roasted',
        'rolled', 'rolling',
        'salted',
        'saved',
        'scored',
        'scrubbed',
        'seasoned',
        'sectioned',
        'seeded',
        'separated',
        'shaved',
        'shelled',
        'shredded',
        'shucked',
        'sifted',
        'smashed',
        'smoked',
        'snipped',
        'sliced',
        'slivered',
        'softened',
        'split',
        'squeezed',
        'steamed',
        'stemmed',
        'soaked',
        'sweetened',
        'thawed',
        'toasted',
        'torn',
        'trimmed',
        'uncored',
        'uncooked',
        'undrained',
        'unhusked',
        'unopened',
        'unpeeled',
        'unsalted',
        'unsweetened',
        'unwashed',
        'use', 'used',
        'warmed',
        'washed',
        'zested',
    ]
    
    other_words_to_filter = [
        # articles:
        'a', 'an', 'the',
        # adverbs:
        'all',
        'approximately',
        'coarsely',
        'crosswise',
        'diagonally', 
        'finely',
        'freshly',
        'horizontally',
        'left',
        'lengthwise', 
        'lightly',
        'like',
        'overnight',
        'piecewise',
        'plus',
        'right',
        'roughly',
        'sideways',
        'slightly',
        'stiffly',
        'then',
        'thickly',
        'thinly',
        'vertically',
        'very',
        'widthwise', 
        # adjectives:
        'according',
        'another',
        'any',
        'certified',
        'classic',
        'coarse', 
        'cold',
        'discarded',
        'dry',
        'extralarge',
        'extrasmall', 
        'fine', 
        'firm',
        'fresh',
        'hard',
        'hot',
        'jumbo',
        'large', 
        'lean',
        'leftover',
        'less',
        'long',
        'medium',
        'more',
        'multicolored',
        'natural',
        'needed',
        'only',
        'optional',
        'organic',
        'original',
        'other',
        'pure',
        'real',
        'rough',
        'short',
        'small',
        'stale',
        'soft',
        'such',
        'thick',
        'thickcut',
        'thicklycut',
        'thicklysliced',
        'thicksliced',
        'thin',
        'thincut',
        'thinlycut',
        'thinlysliced',
        'thinsliced',
        # prepositions:
        'about',
        'across',
        'against',
        'apart',
        'as',
        'at',
        'by',
        'for',
        'from',
        'in',
        'into',
        'of',
        'on',
        'to',
        'with',
        'without',
        # conjunctions:
        'and',
        'if',
        'or',
        # abbreviations:
        'eg', 'ie',
        # nouns:
        'directions',
        'hand',
        'ingredients',
        'list',
        'room',
        'temperature',
        # verbs:
        'add',
        'including',
    ]
    
    #print "Before..."
    #print full_ingred_list
    
    # strip punctuation chars
    # all lowercase
    cleaned_ingred_list = [translate_non_alphanumerics(s).lower() for s in full_ingred_list]
    #print cleaned_ingred_list
    
    # split on whitespace
    # throw out meas units
    #print
    #print "After..."
    ingreds_only = [' '.join(w for w in unidecode.unidecode(ingred_item).split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
         for ingred_item in cleaned_ingred_list
         ]
    ingreds_only = filter(None, ingreds_only) # remove empty strings at end
    #print ingreds_only
    
    brand_names = [
        'a(r)',
        'accent(r)',
        'adams(r)',
        'al fresco(r)',
        'accent(r)',
        'annies lemon chive dressing(r)',
        'archer farms(tm)',
        'backdraft fire sauce(r)',
        'barilla(r)',
        'beau monde (tm)',
        'beechers(r) flagship', 'beechers(r)',
        'beer shiner(r)',
        'best foods(r)',
        'better than bouillon(r)',
        'birds eye(r)',
        'bisquick(r)',
        'bisquick (r)',
        'borden(r)',
        'bragg(r)',
        'briannas home style(r)', 'briannas(r)',
        'brooks(r)',
        'bruces(r)',
        'budweiser(r)',
        'buitoni(r)',
        'bumble bee(r)',
        'bushs best(r)', 'bushs homestyle(r)', 'bushs chili magic(r)', 'bushs(r)',
        'butterball(r)',
        'butter buds(r)',
        'campari(r)',
        'campbells(r) healthy request', 'campbells(r)',
        'carolina pride(r)',
        'carroll shelbys(r)',
        'catelli bistro(r)',
        'cavenders(r)',
        'chambord(r)',
        'chef paul prudhommes salmon magic(r)',
        'cheese whiz(r)', 'cheez whiz(r)',
        'cholula(r)',
        'clamato(r)',
        'classicmac(tm)',
        'classico (tm)',
        'classico (r)',
        'classico(r)',
        'claussen(r)',
        'coke(r)',
        'cointreau(r)',
        'college inn(r)',
        'contadina(r)',
        'cool whip(r)',
        'coors(r)',
        'cornnuts (r)',
        'corona(r)',
        'craisins(r)',
        'crazy steves(tm) cajun cukes', 'crazy steves(r)',
        'crisco(r)',
        'daisy(r)',
        'dei fratelli(r)',
        'del monte(r)',
        'diamond crystal(r)',
        'dickinsons(r)',
        'dole(r)',
        'dole asian island crunch(tm)',
        'dole(r) veggie(tm)',
        'doritos(r)',
        'dubliner(r)',
        'dukes(r)',
        'durkee(r)', 'famous sauce(r)',
        'eagle brand(r)',
        'earth balance(r)',
        'eckrich(r)',
        'edward sons(r)',
        'egg beaters(r)',
        'el pato(r)',
        'embasa(r)',
        'emerald(r)',
        'equal(r)',
        'farmland(r)',
        'fat tire(r)',
        'fiesta blend(r)',
        'follow your heart(r) veganaise(r)', 'follow your heart(r)',
        'foster farms(r)',
        'franks(r) redhot', 'franks red hot (r)', 'franks redhot (r)', 'franks redhot(r)', 'redhot(r)', 'franks(r)',
        'frenchs(r)',
        'frichik(r)',
        'fritos(r)',
        'frontera(r)',
        'goya(r)',
        'gebhardt(r)',
        'girards(r) olde venice',
        'good seasons(r)',
        'gourmet garden(tm)',
        'grapenuts(tm)',
        'great american spice co(tm)',
        'green giant(r)',
        'guinness(r)',
        'harp(r)',
        'hatch(r)',
        'healthy choice(r)',
        'heineken(r)', 
        'heinz (r)', 'heinz(r)',
        "hellmann's(r) best foods(r)", 'hellmanns(r) best foods(r)', 'hellmanns(r)best foods(r)',
        'hellmanns light(r)', 'hellmans(r) light', 'hellmanns(r) low fat',
        'hellmans(r)', "hellmann's(r)",
        'hidden valley ranch(r)', 'hidden valley(r) original ranch(r)',
        'hillshire farm(r)',
        'holland house(r)',
        'horizon(r)',
        'hormel(r)', 'hormel(tm)',
        'house tsang(r)',
        'hunts(r)',
        'idahoan(r)','baby reds(r)', 'buttery homestyle(r)',
        'imagine(r)', 'imagine(tm)',
        'jack daniels(r)',
        'jello(r)',
        'jennieo(r)',
        'jiffy(r)',
        'jim beam(r)',
        'jimmy dean(r)',
        'johnsonville(r)',
        'johnnys seasoning salt(r)',
        'kame(r)',
        'karys(r)',
        'kc masterpiece(r)',
        'keebler club(r)', 'keebler(r)',
        'keens(r)',
        'kens steak house(r)',
        'kens steak house lite northern italian dressing(r)',
        'kerrygold(r)',
        'kewpie(r)',
        'king arthur(r)',
        'king oscar(r)',
        'kirkland(r)',
        'kitchen basics(r)',
        'kitchen bouquet(r)',
        'kitchen secrets(r)',
        'klondike gourmet(r)', 'klondike goldust(r)', 'klondike(r)',
        'knorr(r) rice sides(tm)', 'knorr(r) fiesta sides(tm)', 'knorr(r)', 'knorrs(r)',
        'knox (r)',
        'kraft(r)',
        'laphroiag(r)',
        'laughing cow(r)',
        'lawrys(r)',
        'libbys(r)',
        'lightlife(r) organic smoky tempeh strips(r)', 'lightlife(r)',
        'lipton(r) recipe secrets(r)', 'lipton(r)',
        'locatelli(r)',
        'loma linda(r)',
        'lotus(r)',
        'louisiana(r)',
        'maggi(r)',
        'mahatma(r)',
        'maifun(r)',
        'maille(r)',
        'malibu(r)',
        'marie callenders(r)',
        'maries(r)',
        'market pantry(tm)',
        'marzettis(r)',
        'maui(r)',
        'mazola(r)',
        'mccormick chili seasoning mix sodium(r)', 'mccormick(r)',
        'mexene(r)',
        'mexenes(r)',
        'mexicorn(r)',
        'miller lite(r)',
        'mimiccreme(r)',
        'minute(r)',
        'miracle whip(r)',
        'miracle whip(tm)',
        'miracle whip (tm)',
        'miracle whip free(r)',
        'miracle whip light(r)', 'light miracle whip(r)',
        'montreal steak seasoning(r)',
        'morningstar farms(r) grillers recipe crumbles(r)', 'morningstar farms(r) recipe crumbles(r)', 'morningstar farms(r)',
        'morton(r) natures seasons(r)', 'morton(r)',
        'motts(r)',
        'mountain dew(r)',
        'mrs butterworths(r)',
        'mrs dash(r)',
        'muir glen(r)',
        'nabisco(r)',
        'natural goodness(tm)',
        'negra modelo(r)',
        'newcastle(r)',
        'newmans own(r)',
        'newmans own(r) lighten up(r)', 'lighten up(r)',
        'niblets(r)',
        'nissin(r) top ramen',
        'no yolks(r)',
        'ocean spray(r)',
        'old bay(r)', 'old bay (tm)', 'old bay(tm)',
        'old el paso(r)',
        'olive garden(r)',
        'oreida(r) steam n mash(r)', 'oreida(r)', 'ore ida(r)',
        'oreo(r)',
        'ortega(r)',
        'ottogi ramyonsari(r)', 'ottogi(r)',
        'quorn(tm)',
        'pace(r)',
        'pam(r)',
        'pataks(r)',
        'penzeys(r)', 'penzys(r)',
        'pepperidge farm(r)',
        'perdue(r)',
        'pernod(r)',
        'perrins(r)',
        'pillsbury grands(r)',
        'planters(r)',
        'plochmans(r)',
        'progresso(r)',
        'ragu(r) old world style(r)', 'ragu(r)',
        'rapunzel(r)',
        'ready rice(r)',
        'realime(r)',
        'red gold(r)',
        'renees(r)',
        'reynolds wrap(r)', 'reynolds(r)',
        'rice sides(tm)',
        'ronzoni(r)',
        'rotel(r)',
        'saco(r)',
        'sambazon',
        'samuel smith(r)',
        'sargento(r)',
        'sason accent(r)',
        'sb(r) golden curry', 'sb(r)',
        'seasonall(r)',
        'shiner bock(r)',
        'silk(r)',
        'simple crisp(tm)',
        'simply balanced(tm)',
        'simply potatoes(r)',
        'slap ya mama(r)',
        'smithfield(r)',
        'snack factory(r)',
        'snickers(r)',
        'so delicious(r)',
        'southern comfort (r)',
        #'spam(r)',
        'spectrum(r)',
        'spice islands(r)',
        'spike(r)',
        'splenda(r)',
        'sriracha(r)',
        'steam whistle pilsner(r)',
        'sunsweet(r) dnoir(tm)',
        'sutton dodge(tm)',
        'swanson(r) vegetable flavor boost(r)', 'swanson(r)', 'natural goodness(r)',
        'sweet baby rays(r)',
        'sweetn low(r)',
        'tabasco(r)', 'tabasco(tm)',
        'taco bell(r)',
        'tajin(r)',
        'tapatio(r)',
        'tater tots(r)',
        'texas pete(r)', 'texas petess(r)',
        'thai kitchen(r)',
        'three bridges(r)',
        'tony chacheres(r)',
        'top ramen(r)',
        'trader joes(r)',
        'truroots(r)',
        'truvia(r)',
        'uncle bens (r)', 'uncle bens(r)',
        'v(r)', # what remains from "V8"
        'valentina(r)',
        'vegenaise(r)',
        'veggie(tm)',
        #'velveeta(r)',
        'vidalia(r)',
        'vietti(r)',
        'vindaloo sauce maya kaimal(r)', 'vindaloo(r)',
        'voskos(r)',
        'wesson(r)',
        'white lily(r)',
        'wishbone(r)',
        'wondra(r)',
        'yoplait(r)',
        'yuengling(r)',
        'yves veggie cuisine(r)',
        'zatarains(r)',
        'zing zang(r)',
    ]
    
    def remove_brands(ingred_str):
        str_without_brand = ingred_str
        for bn in brand_names:
            str_without_brand = str_without_brand.replace(bn, '')
        return str_without_brand
    
    ingreds_without_brands = [' '.join(remove_brands(ingred_str).strip().split()) for ingred_str in ingreds_only]
    
    return ingreds_without_brands

# def ascii_encode(text):
#     return text.encode("ascii", "ignore")

extract_ingredients(df[['ingredients']].values[867][0])

['recipe pastry double crust pie',
 'white sugar',
 'allpurpose flour',
 'salt',
 'water',
 'cranberries',
 'lemon juice',
 'cinnamon',
 'butter']

In [100]:
# print len(df[['ingredients']].values)
# print df[['ingredients']].values.shape
# print type(df[['ingredients']].values)
# #print df[['ingredients']].values
# print 

# print len(df[['ingredients']].values[867])
# print df[['ingredients']].values[867].shape
# print type(df[['ingredients']].values[867])
# #print df[['ingredients']].values[867]
# print

# #df[['ingredients']].values[][0]

# print len(df[['ingredients']].values[867][0])
# #print df[['ingredients']].values[867][0].shape
# print type(df[['ingredients']].values[867][0])
# #print df[['ingredients']].values[867][0]
# print

# print len(np.squeeze(df[['ingredients']].values))
# print np.squeeze(df[['ingredients']].values).shape
# print type(np.squeeze(df[['ingredients']].values))
# print np.squeeze(df[['ingredients']].values)
# print 

# print len(np.squeeze(df[['ingredients']].values)[867])
# #print np.squeeze(df[['ingredients']].values).shape
# print type(np.squeeze(df[['ingredients']].values)[867])
print np.squeeze(df[['ingredients']].values)[867]
print 

#print np.squeeze(df[['ingredients']].values)[867][2].encode("ascii", "ignore")

[u'1 recipe pastry for a 9 inch double crust pie', u'1 1/2 cups white sugar', u'1/3 cup all-purpose flour', u'1/4 teaspoon salt', u'1/2 cup water', u'1 (12 ounce) package fresh cranberries', u'1/4 cup lemon juice', u'1 dash ground cinnamon', u'2 teaspoons butter', u'Add all ingredients to list', u'Add all ingredients to list']



In [101]:
ingreds_arr_of_lists = np.squeeze(df[['ingredients']].values)
vect_extract_ingred = np.vectorize(extract_ingredients, otypes=[list])

processed_ingreds_arr_of_lists = vect_extract_ingred(ingreds_arr_of_lists)

#vect_ascii_encode = np.vectorize(ascii_encode, otypes=[list])
#processed_ingreds_arr_of_lists = vect_ascii_encode(processed_ingreds_arr_of_lists)

print type(processed_ingreds_arr_of_lists)
print len(processed_ingreds_arr_of_lists)
print processed_ingreds_arr_of_lists.size



<type 'numpy.ndarray'>
10095
10095


In [102]:
print processed_ingreds_arr_of_lists
#unique_vocab = set(np.hstack(processed_ingreds_arr_of_lists))

[ ['grain wild rice mix', 'blackeyed peas', 'red onion', 'green bell pepper', 'crabmeat picked over cartilage shell fragments', 'parsley', 'lemon juice', 'extravirgin olive oil', 'pepper sauce', 'salt', 'black pepper', 'tomato']
 ['broccoli', 'red onion', 'bacon', 'raisins', 'almonds', 'mayonnaise', 'white sugar', 'white wine vinegar']
 ['sesame seeds', 'bow tie pasta', 'vegetable oil', 'light soy sauce', 'rice vinegar', 'sesame oil', 'white sugar', 'ginger', 'black pepper', 'chicken breast meat', 'cilantro', 'green onion']
 ...,
 ['boneless skinless chicken breast', 'steak sauce', 'fusilli pasta', 'chicken bouillon', 'vidalia onion', 'avocados', 'cherry tomatoes', 'ranchstyle salad dressing']
 ['tomatoes', 'sweet onion', 'olive oil', 'oregano', 'basil', 'parsley', 'italian bread', 'parmesan cheese']
 ['bonein chicken breast', 'chicken broth', 'white onion', 'garlic', 'salt', 'tomatillos', 'serrano peppers', 'white onion', 'garlic', 'salt', 'corn tortillas', 'vegetable oil', 'queso fre

In [103]:
flattened_ingred_list = np.hstack(processed_ingreds_arr_of_lists)
print len(flattened_ingred_list)
print flattened_ingred_list

unique_vocab = sorted(list(set(flattened_ingred_list)))
print len(unique_vocab)
print unique_vocab

105522
['grain wild rice mix' 'blackeyed peas' 'red onion' ..., 'queso fresco'
 'white onion' 'cilantro']
6545
['', 'acai pulp (r)', 'achiote annatto seeds', 'achiote powder', 'acini de pepe pasta', 'acini di pepe pasta', 'acorn squash', 'acorn squashes', 'active yeast', 'additional cheddar cheese', 'additional cocktail sauce dipping', 'additional green onions', 'adobo sauce', 'adobo sauce chilies', 'adobo sauce chipotle peppers', 'adobo sauce chipotle peppers goy', 'adobo seasoning', 'adobo seasoning cumin', 'adobo seasoning rojo adobo', 'adzuki beans', 'agave nectar', 'agave syrup', 'aged balsamic vinegar', 'ahi tuna', 'ahi tuna steaks', 'ahi yellowfin tuna steak', 'alaskan cod', 'alaskan king crab legs shell', 'albacore tuna', 'albacore tuna water', 'alder wood', 'ale', 'aleppo pepper', 'alfalfa sprouts', 'alfredo pasta sauce', 'alfredo sauce', 'alfredostyle pasta sauce', 'allbeef dog', 'alligator meat', 'allpurpose baking mix', 'allpurpose flour', 'allpurpose potatoes', 'allspice',

In [104]:
# find ingredients that contain brand names
# these are identifiable from the (r) or (tm) in the names, which unidecode puts there in place of \u00ae or \u2122
brand_name_ingreds = [s for s in unique_vocab if '(r)' in s or '(tm)' in s]
print len(brand_name_ingreds)
#print brand_name_ingreds
for brand_name in brand_name_ingreds:
    print brand_name

21
acai pulp (r)
cheese food velveeta (r)
chicken broth chicken broth goodness(r) chicken broth
chicken broth goodness(r) chicken broth chicken broth
chicken broth regular goodness(tm)
chili seasoning mix frenchs chilio seasoning mix(r)
farina cereal cream wheat(r)
granular nocalorie sucralose sweetener splenda (r)
hash brown potatoes oreid steam n mash(r) garlic potatoes
hidden valley(r) ranch(r) salad dressing seasoning mix
lowsodium chicken broth goodness(r)
mayonnaise hellmanns(r)
old bay seasoning(tm)
pepper sauce franks red hot(r)
pretzel crisps(r)
ranch salad dressing hidden valley(r) ranch(r)
sauce (r)
sauce franks red (r)
spam(tm) luncheon meat
stevia raw(r) bakers
tempeh bacon smoky tempeh strips(r)


In [105]:
flat_ingred_arr = [' '.join(lst) for lst in ingreds_arr_of_lists]

cv = CountVectorizer(vocabulary=unique_vocab)
recipe_ingredient_matrix = cv.fit_transform(flat_ingred_arr)

print type(recipe_ingredient_matrix)
print recipe_ingredient_matrix.size

print recipe_ingredient_matrix

<class 'scipy.sparse.csr.csr_matrix'>
107576
  (0, 1637)	1
  (0, 2503)	1
  (0, 3283)	1
  (0, 3785)	1
  (0, 3941)	1
  (0, 3975)	1
  (0, 4103)	1
  (0, 4189)	1
  (0, 4211)	3
  (0, 4918)	1
  (0, 5104)	1
  (0, 5142)	1
  (0, 5891)	1
  (1, 52)	1
  (1, 280)	1
  (1, 792)	1
  (1, 3630)	1
  (1, 3975)	1
  (1, 4638)	1
  (1, 5622)	1
  (1, 6212)	1
  (2, 1141)	1
  (2, 1390)	1
  (2, 2397)	1
  (2, 3637)	1
  :	:
  (10092, 4117)	1
  (10092, 5061)	1
  (10092, 5142)	1
  (10092, 5565)	1
  (10092, 5913)	1
  (10093, 350)	1
  (10093, 766)	1
  (10093, 1101)	1
  (10093, 3941)	1
  (10093, 3975)	1
  (10093, 4032)	1
  (10093, 4084)	1
  (10093, 4103)	1
  (10093, 5913)	1
  (10094, 810)	1
  (10094, 1141)	2
  (10094, 1390)	1
  (10094, 1565)	1
  (10094, 2324)	2
  (10094, 3941)	1
  (10094, 3975)	3
  (10094, 4231)	1
  (10094, 5104)	2
  (10094, 5890)	1
  (10094, 5983)	1


In [106]:
svd = TruncatedSVD(n_components=50)
reduced_recipe_ingredient_matrix = svd.fit_transform(recipe_ingredient_matrix)

print type(reduced_recipe_ingredient_matrix)
print reduced_recipe_ingredient_matrix.size

<type 'numpy.ndarray'>
504750


In [107]:
df_merged['ingredients'] = df_merged['ingredients'].apply(extract_ingredients)
df_merged.sample(5)

NameError: name 'df_merged' is not defined

In [108]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

def makePipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', KMeans(n_clusters=2, init='k-means++', max_iter=100, n_init=1))
    ])
    return pipeline

def trainModel(X, Y):
    model = makePipeline()
    model.fit(X, Y)
    return model

X = [' '.join(f) for f in df_merged['ingredients'].values]
Y = df_merged['category'].values

model = trainModel(X, Y)

NameError: name 'df_merged' is not defined

In [58]:
true_k = len(np.unique(Y))
true_k

2

In [65]:
preds = model.predict(X)

[[2263 8019]
 [2984  269]]


In [72]:
uniq = np.unique(Y)
pred_labeled = [uniq[1] if p == 0 else uniq[0] for p in preds]
print confusion_matrix(pred_labeled, Y)

[[2984  269]
 [2263 8019]]


In [73]:
from sklearn.metrics import classification_report
print classification_report(pred_labeled, Y)

             precision    recall  f1-score   support

   bbqbread       0.57      0.92      0.70      3253
     salads       0.97      0.78      0.86     10282

avg / total       0.87      0.81      0.82     13535



In [59]:
print("Top terms per cluster:")

vectorizer = model.named_steps['vect']
km = model.named_steps['clf']
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

terms = vectorizer.get_feature_names()
for i in range(true_k):
    print "Cluster %d:" % i
    for ind in order_centroids[i, :10]:
        print ' %s' % terms[ind]
    print


Top terms per cluster:
Cluster 0:
 pepper
 cheese
 garlic
 salt
 black
 oil
 onion
 red
 sauce
 green

Cluster 1:
 flour
 baking
 sugar
 allpurpose
 white
 milk
 butter
 powder
 salt
 eggs



In [110]:
def syntax_text(text):
    """Detects syntax in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    tokens = client.analyze_syntax(document).tokens

    # part-of-speech tags from enums.PartOfSpeech.Tag
    pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
               'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')

    for token in tokens:
        print(u'{}: {}'.format(pos_tag[token.part_of_speech.tag],
                               token.text.content))

In [111]:
syntax_text('three cored and peeled green tomatoes')

NUM: three
VERB: cored
CONJ: and
VERB: peeled
ADJ: green
NOUN: tomatoes


In [115]:
ingred_list = [u'3 avocados - peeled, pitted, and mashed',
 u'1 lime, juiced',
 u'1 teaspoon salt',
 u'1/2 cup diced onion',
 u'3 tablespoons chopped fresh cilantro',
 u'2 roma (plum) tomatoes, diced',
 u'1 teaspoon finely minced garlic',
 u'1 pinch ground cayenne pepper (optional)',
 u'Add all ingredients to list',
 u'Add all ingredients to list']

for ingred_str in ingred_list:
    syntax_text(ingred_str)
    print "-----"

NUM: 3
NOUN: avocados
PUNCT: -
VERB: peeled
PUNCT: ,
VERB: pitted
PUNCT: ,
CONJ: and
ADJ: mashed
-----
NUM: 1
NOUN: lime
PUNCT: ,
VERB: juiced
-----
NUM: 1
NOUN: teaspoon
NOUN: salt
-----
NUM: 1
X: /
NUM: 2
NOUN: cup
NOUN: diced
NOUN: onion
-----
NUM: 3
NOUN: tablespoons
VERB: chopped
ADJ: fresh
NOUN: cilantro
-----
X: 2
NOUN: roma
PUNCT: (
NOUN: plum
PUNCT: )
NOUN: tomatoes
PUNCT: ,
VERB: diced
-----
NUM: 1
NOUN: teaspoon
ADV: finely
VERB: minced
NOUN: garlic
-----
NUM: 1
NOUN: pinch
NOUN: ground
NOUN: cayenne
NOUN: pepper
PUNCT: (
ADJ: optional
PUNCT: )
-----
VERB: Add
DET: all
NOUN: ingredients
PRT: to
VERB: list
-----
VERB: Add
DET: all
NOUN: ingredients
PRT: to
VERB: list
-----
